In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup

In [2]:
def trowDataScrapper(tr_element): #bir satır alır ve verisini json olarak döndürür.
    td_elements = tr_element.find_all('td')

    kübDoc = td_elements[5].find('a', class_='badge')
    hrefKübDoc = kübDoc['href'] if kübDoc else None

    ktDoc = td_elements[6].find('a', class_='badge')
    hrefKtDoc = ktDoc['href'] if ktDoc else None

    trowDataJson = {
    "ilaçAdı":td_elements[0].text.strip(),
    "etkinMaddeAdı":td_elements[1].text.strip(),
    "firmaAdı":td_elements[2].text.strip(),
    "kübOnayTarihi":td_elements[3].text.strip(),
    "ktOnayTarihi":td_elements[4].text.strip(),
    "kübDocLink":hrefKübDoc,
    "ktDocLink":hrefKtDoc
    }

    return trowDataJson

    

In [3]:
def tbodyDataScrapper(tbodyHtml): #tbody --> split tr-> trowscrapper
    soup = BeautifulSoup(tbodyHtml, 'html.parser')
    tr_elements = soup.find_all('tr')
    #scraped_data = list(map(lambda tr_element: trowDataScrapper(tr_element), tr_elements))
    scraped_data = []

    for tr_element in tr_elements:
        json_data = trowDataScrapper(tr_element)
        scraped_data.append(json_data)

    return scraped_data
    

In [ ]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 20)
maxPageNum = 148
data=[]

try:
    driver.get("https://www.titck.gov.tr/kubkt")
    postsLengthElement = wait.until(EC.presence_of_element_located((By.NAME, "posts_length")))

    time.sleep(3)

    selectLength = Select(postsLengthElement)
    selectLength.select_by_value("100") 
    time.sleep(3)

    
    while True: 
        currentPageNum = int(driver.find_element(By.CLASS_NAME, "paginate_button.current").text)
        
        next_button = driver.find_element(By.ID, "posts_next")
        driver.execute_script("arguments[0].scrollIntoView();", next_button)  


        print(f"{currentPageNum} verisi çekiliyor.")
        tbody_element = driver.find_element(By.TAG_NAME, "tbody")
        tbodyHtml = tbody_element.get_attribute('outerHTML')
        pageData=tbodyDataScrapper(tbodyHtml)
        print(len(pageData))

        data.extend(pageData)

        if currentPageNum == maxPageNum: # son sayfada veriyi çektikten sonra sonlandır.
            break
       
        driver.execute_script("arguments[0].click();", next_button)
        WebDriverWait(driver, 10).until(
            EC.staleness_of(next_button)  # Sayfa yenilenecek ve buton eski olacak
        )


finally:
    print("closing")
    driver.quit()

1 verisi çekiliyor.
100
2 verisi çekiliyor.
100
3 verisi çekiliyor.
100
4 verisi çekiliyor.
100
5 verisi çekiliyor.
100
6 verisi çekiliyor.
100
7 verisi çekiliyor.
100
8 verisi çekiliyor.
100
9 verisi çekiliyor.
100
10 verisi çekiliyor.
100
11 verisi çekiliyor.
100
12 verisi çekiliyor.
100
13 verisi çekiliyor.
100
14 verisi çekiliyor.
100
15 verisi çekiliyor.
100
16 verisi çekiliyor.
100
17 verisi çekiliyor.
100
18 verisi çekiliyor.
100
19 verisi çekiliyor.
100
20 verisi çekiliyor.
100
21 verisi çekiliyor.
100
22 verisi çekiliyor.
100
23 verisi çekiliyor.
100
24 verisi çekiliyor.
100
25 verisi çekiliyor.
100
26 verisi çekiliyor.
100
27 verisi çekiliyor.
100
28 verisi çekiliyor.
100
29 verisi çekiliyor.
100
30 verisi çekiliyor.
100
31 verisi çekiliyor.
100
32 verisi çekiliyor.
100
33 verisi çekiliyor.
100
34 verisi çekiliyor.
100
35 verisi çekiliyor.
100
36 verisi çekiliyor.
100
37 verisi çekiliyor.
100
38 verisi çekiliyor.
100
39 verisi çekiliyor.
100
40 verisi çekiliyor.
100
41 verisi

In [8]:
import pandas as pd
df = pd.DataFrame(data)

In [10]:
df.head()

,ilaçAdı,etkinMaddeAdı,firmaAdı,kübOnayTarihi,ktOnayTarihi,kübDocLink,ktDocLink
0,% 0.4 LİDODEKS % 5 DEKSTROZ İÇİNDE İ.V. İNFÜZY...,LİDOKAİN HİDROKLORÜR,POLİFARMA İLAÇ SAN. VE TİC. A.Ş.,05/08/2016,05/08/2016,https://titck.gov.tr/storage/kubKtAttachments/...,https://titck.gov.tr/storage/kubKtAttachments/...
1,% 0.8 LİDODEKS % 5 DEKSTROZ İÇİNDE İ.V. İNFÜZY...,LİDOKAİN HİDROKLORÜR,POLİFARMA İLAÇ SAN. VE TİCARET A.Ş.,05/08/2016,05/08/2016,https://titck.gov.tr/storage/kubKtAttachments/...,https://titck.gov.tr/storage/kubKtAttachments/...
2,% 10 DEKSTROZ BIOSEL SUDAKI HIPERTONIK SOLÜSYONU,DEKSTROZ,OSEL ILAÇ SAN. VE TIC.A.Ş.,06/02/2015,06/02/2015,https://titck.gov.tr/storage/kubKtAttachments/...,https://titck.gov.tr/storage/kubKtAttachments/...
3,% 10 DEKSTROZ BIOSEL SUDAKI HIPERTONIK SOLÜSYONU,DEKSTROZ,OSEL ILAÇ SAN. VE TIC.A.Ş.,06/02/2015,06/02/2015,https://titck.gov.tr/storage/kubKtAttachments/...,https://titck.gov.tr/storage/kubKtAttachments/...
4,% 10 DEKSTROZ BIOSEL SUDAKI HIPERTONIK SOLÜSYONU,DEKSTROZ,OSEL ILAÇ SAN. VE TIC.A.Ş.,06/02/2015,06/02/2015,https://titck.gov.tr/storage/kubKtAttachments/...,https://titck.gov.tr/storage/kubKtAttachments/...


In [11]:
df.to_csv("data/output.csv", index=False, encoding='utf-8-sig')